In [1]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [9]:
def resize_image(input_path, size=(224, 224)):
    # Abrir la imagen
    image = Image.open(input_path).convert("L")  # Convertir a escala de grises
    
    # Obtener tamaño original
    original_width, original_height = image.size
    aspect_ratio = original_width / original_height
    
    # Calcular nuevo tamaño
    if aspect_ratio > 1:
        # Imagen más ancha que alta
        new_width = size[0]
        new_height = int(new_width / aspect_ratio)
    else:
        # Imagen más alta que ancha
        new_height = size[1]
        new_width = int(new_height * aspect_ratio)
    
    # Redimensionar la imagen manteniendo la relación de aspecto
    # si la imagen es mas chica que el tamaño deseado, uso interpolación LANCZOS, sino LINEAL
    if original_width < size[0] or original_height < size[1]:
        resized_image = image.resize((new_width, new_height), Image.LANCZOS)
    else:
        resized_image = image.resize((new_width, new_height), Image.BILINEAR)
    
    # Crear un nuevo fondo de 224x224 y pegar la imagen redimensionada en el centro
    new_image = Image.new("L", size)
    new_image.paste(resized_image, ((size[0] - new_width) // 2, (size[1] - new_height) // 2))
    
    return new_image

In [15]:
# show original image
image = Image.open("CelebA/celebA/img_align_celeba/img_align_celeba/125478.jpg")
image.show()

# image size
print("original image size:", image.size)

# resize image
resized_image = resize_image("CelebA/celebA/img_align_celeba/img_align_celeba/125478.jpg", size=(224, 224))
resized_image.show()

# resized image size
print("resized image size:", resized_image.size)

original image size: (178, 218)
resized image size: (224, 224)


In [16]:
# save resized image as jpg
resized_image.save("125478_resized.jpg")

In [10]:
# probar de redimensionar la imagen a 112x112
resized_image = resize_image("CelebA/celebA/img_align_celeba/img_align_celeba/125478.jpg", size=(112, 112))
resized_image.show()